Calcula la suma y multiplicación de las velocidades para cada muestra en diferentes cortes de eventos

In [ ]:
import pandas as pd
from pathlib import Path
import os

def calculate(filename):
    directory = os.path.join(Path().absolute(), "speed-prediction-logs")
    path = os.path.join(directory, "TimeWindow")
    path = os.path.join(path, filename + ".csv")
    df = pd.read_csv(path)

    df = df.fillna(0)
    
    df["sum" + filename] = df["avgSpeedPerTest"] + df["avgIntratecla"] + df["avgScrollSpeed"]
    df["mul" + filename] = df["avgSpeedPerTest"] * df["avgIntratecla"] * df["avgScrollSpeed"]

    column_names = df.columns.values.tolist()
    column_names.remove('name')
    column_names.remove("sum" + filename)
    column_names.remove("mul" + filename)
    for name in column_names:
        df.pop(name)
    
    return df

directory = os.path.join(Path().absolute(), "speed-prediction-logs")
path = os.path.join(directory, "TimeWindow")
path = os.path.join(path, str(120000) + ".csv")
df = pd.read_csv(path)

logs = range(5000,121000,5000)
for name in logs:
    df = df.merge(right=calculate(str(name)), on='name', how='left')

speed_df = df.copy(deep=True)

for name in logs:
    name = str(name)
    df["sum"+name] = df["sum"+name]  / df["sum120000"]
    df["mul"+name] = df["mul"+name]  / df["mul120000"]

speed_df_norm = df
#speed_df_norm.to_csv('speed_prediction_normalized.csv')

In [ ]:
speed_df

Observar los números calculados por suma

In [ ]:
def remove_columns_that_starts_with(df, prefix):
    column_names = df.columns.values.tolist()
    for name in column_names:
        if name.startswith(prefix):
            df.pop(name)
    return df

speed_df_only_sum = speed_df.copy(deep=True)
speed_df_only_sum = remove_columns_that_starts_with(speed_df_only_sum, "mul")
speed_df_only_sum

In [ ]:
speed_df_only_sum.describe()

Ver cuanto se acercan los valores de la suma al final

In [ ]:
df_test = speed_df_only_sum.copy(deep=True)
column_names = df_test.columns.values.tolist()
for name in column_names:
    if "120000" in name:
        continue
    if name.startswith("sum"):
        amount = name.replace("sum","")
        amount = int(amount)
        df_test.loc[amount > df["timePerTest"], name] = None

        df_test[name] = df_test[name] / df_test["sum120000"]

def remove_columns_that_dont_have(df, word):
    column_names = df.columns.values.tolist()
    for name in column_names:
        if word not in name:
            df.pop(name)
    return df

df_test = remove_columns_that_dont_have(df_test, "sum")
df_test

In [ ]:
df_test.describe()